In [1]:
from gerrychain import (Graph, Partition, Election, MarkovChain,
                        proposals, updaters, constraints, accept)
from gerrychain.updaters import Tally, cut_edges
from functools import partial
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import numpy as np

## Upload state information

In [3]:
basic_st_info = pd.read_csv('state_data.csv')

In [4]:
exclude = ['Alaska', 'Delaware', 'Hawaii', 'Idaho', 'Kentucky', 'Maine', 'Montana', 
           'New Hampshire', 'North Dakota', 'Rhode Island', 'South Dakota', 'Vermont', 
           'West Virginia', 'Wyoming', 'California', 'Oregon', 'Massachusetts']

In [5]:
select = [(name in exclude) for name in basic_st_info['name']]
to_drop = basic_st_info.loc[select].index
basic_st_info = basic_st_info.drop(to_drop)
basic_st_info = basic_st_info.reset_index(drop=True)

In [7]:
st_data = {}
for i, st in enumerate(states):
    info = {}
    info['name'] = basic_st_info.loc[i, 'name']
    fips = basic_st_info.loc[i, 'fips']
    if fips < 10:
        info['fips'] = '0' + str(fips)
    else:
        info['fips'] = str(fips)
    st_data[st] = info

## Functions to process data

In [333]:
def clean_extra_rows(dat):
    first_index = 0
    for geoid in dat["GEOID20"]:
        if geoid[0] == "B":
            break
        else:
            first_index += 1
    
    orig_length = len(dat)
    weird_rows = dat.loc[first_index:orig_length]
    
    dat = dat.drop(list(range(first_index, orig_length)))
    
    curr = first_index
    while curr < orig_length:
        new_id = weird_rows.loc[curr, 'FriendlyId'][0:11]
        weird_rows.loc[curr, 'GEOID20'] = new_id
        while (curr+1) < orig_length:
            if weird_rows.loc[curr+1, 'FriendlyId'][0:11] == new_id:
                weird_rows = weird_rows.drop(curr+1)
                curr += 1
            else:
                break
        curr += 1
    
    dat = dat.append(weird_rows, ignore_index=True)
    return dat

In [334]:
def index_of_dups(array):
    seen = set()
    index = set()
    for i, elem in enumerate(array):
        if elem in seen:
            index.add(i)
        else:
            seen.add(elem)
    return index

In [335]:
def fix_Florida(dat, census):
    unique_ids = set()
    for i, geoid in enumerate(dat['GEOID20']):
        if not census['GEOID20'].str.contains(geoid).any():
            if geoid[0:11] not in unique_ids:
                unique_ids.add(geoid[0:11])
                dat.loc[i, 'GEOID20'] = geoid[0:11]
            else:
                dat = dat.drop(i)
                
    dat = dat.reset_index(drop=True)
    dat = dat.drop_duplicates(subset=['GEOID20'])
    dat = dat.reset_index(drop=True)
    return dat

In [336]:
def get_map(state, kind, ret_census):
    try:
        # load state census data (geometry for vtds)
        f_name = "tl_2020_" + st_data[state]['fips'] + "_vtd20"
        census = gpd.read_file(f_name + "/" + f_name + ".shp")

        # load data for state's most proportionally representative map
        dat = pd.read_csv(kind + "_maps/" + st_data[state]['fips'] + ".csv")
        dat = dat.astype({'GEOID20': str, 'FriendlyId': str})
        
        # clear weird rows at the end of the dataframe
        if len(dat) != len(census):
            dat = clean_extra_rows(dat)

        if state == 'FL':
            dat = fix_Florida(dat, census)
            
        if len(dat) != len(census):
            raise Exception("Error! length of dataset != length of census")
        
        if ret_census:
            return dat, census
        else:
            return dat
    
    except Exception as e:
        print(state, "\b:", e)

## Upload proposed maps

In [12]:
for state in states:
    st_data[state]['proposed'] = get_map(state, 'proposed', False)

## Add actual wins and EGs to state data

In [13]:
state_results = {}
for state in states:
    state_results[state] = {}

In [14]:
def get_actual_wins(state):
    df = st_data[state]['proposed']
    dem_wins = 0
    rep_wins = 0
    district_nums = pd.unique(df['District'])
    for district in district_nums:
        d_rows = df.loc[df['District'] == district]
        dem_votes = sum(d_rows['Dem_2020_Pres'])
        rep_votes = sum(d_rows['Rep_2020_Pres'])
        if dem_votes > rep_votes:
            dem_wins += 1
        elif rep_votes > dem_votes:
            rep_wins += 1
            
    return dem_wins, rep_wins

In [15]:
def get_efficiency_gap(state):
    df = st_data[state]['proposed']
    total_wasted_rep = 0
    total_wasted_dem = 0
    
    district_nums = pd.unique(df['District'])
    for district in district_nums:
        d_rows = df.loc[df['District'] == district]
        dem = sum(d_rows['Dem_2020_Pres'])
        rep = sum(d_rows['Rep_2020_Pres'])
        total = dem + rep
        if dem > rep:
            total_wasted_rep += rep
            total_wasted_dem += dem - int(0.51 * total)
        elif rep > dem:
            total_wasted_dem += dem
            total_wasted_rep += rep - int(0.51 * total)
    total_votes = sum(df['Total_2020_Pres'])
    
    eg = (total_wasted_dem - total_wasted_rep) / total_votes
    return eg

In [529]:
def eg_to_seats(state, eg):
    df = st_data[state]['proposed']
    num_districts = len(pd.unique(df['District']))
    return eg * num_districts

In [530]:
for state in states:
    try:
        dem, rep = get_actual_wins(state)
        st_data[state]['D_actual'] = dem
        st_data[state]['R_actual'] = rep
        
        eg = get_efficiency_gap(state)
        state_results[state]['eg'] = eg
        eg_seats = eg_to_seats(state, eg)
        state_results[state]['eg_seats'] = eg_seats
        
    except Exception as e:
        print(state, '\b:', e)

## Run MCMC

In [20]:
import logging
import threading
import time

In [286]:
def make_gdf(census, dat):
    vtd_shapes = gpd.GeoSeries()
    for i, geoid in enumerate(dat['GEOID20']):
        # shape associated with current geoid
        shape = census.loc[census['GEOID20'] == geoid, 'geometry']
        vtd_shapes = vtd_shapes.append(shape, ignore_index=True)
    gdat = gpd.GeoDataFrame(dat, geometry=vtd_shapes)
    return gdat

In [289]:
def run_Markov_chain(state):
    logging.info("Thread %s: starting", state)
    
    try:
        prop_rep, census = get_map(state, 'prop_rep', True)
        gdat = make_gdf(census, prop_rep)

        # begin performing gerrychain
        dual_graph = Graph.from_geodataframe(gdat)

        election_updaters = {"PRES20": Election("PRES20", {"Democratic": "Dem_2020_Pres", "Republican": "Rep_2020_Pres"})}

        my_updaters = {"population": Tally("Total_2020_Total", alias="population"),
                       "cut_edges": cut_edges}
        my_updaters.update(election_updaters)

        initial_partition = Partition(
            dual_graph,
            assignment='District',
            updaters=my_updaters
        )

        ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)
        proposal = partial(proposals.recom,
                           pop_col="Total_2020_Total",
                           pop_target=ideal_population,
                           epsilon=0.02,
                           node_repeats=2
                          )

        compactness_constraint = constraints.UpperBound(
            lambda p: len(p["cut_edges"]),
            2*len(initial_partition["cut_edges"])
        )

        chain = MarkovChain(
            proposal=proposal,
            constraints=[
                compactness_constraint
            ],
            accept=accept.always_accept,
            initial_state=initial_partition,
            total_steps=5000
        )
        
        D_wins20 = []
        R_wins20 = []
        start_time = time.time()

        for i, current_partition in enumerate(chain):
            D_wins20.append(current_partition["PRES20"].wins("Democratic"))   # track number of dem wins in 2020
            R_wins20.append(current_partition["PRES20"].wins("Republican"))   # track number of rep wins in 2020

        print((state + " --- %.2f minutes ---") % ((time.time() - start_time) / 60))
            
        state_results[state]['D_wins20'] = D_wins20
        state_results[state]['R_wins20'] = R_wins20
        
    except Exception as e:
        print(state, "\b:", e)
        print()
    
    logging.info("Thread %s: finishing", state)

### Code for threading adapted from: https://realpython.com/intro-to-python-threading/

In [ ]:
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

threads = list()
for state in states:
    logging.info("Main    : create and start thread %d.", state)
    x = threading.Thread(target=run_Markov_chain, args=(state,))
    threads.append(x)
    x.start()

for index, thread in enumerate(threads):
    logging.info("Main    : before joining thread %d.", states[index])
    thread.join()
    logging.info("Main    : thread %d done", states[index])

## Save results of MCMC

In [268]:
D_results = pd.DataFrame(index=list(range(5000)))
R_results = pd.DataFrame(index=list(range(5000)))

In [269]:
for state in states:
    D_results[state] = state_results[state]['D_wins20']
    R_results[state] = state_results[state]['R_wins20']

In [270]:
D_results.to_csv('D_results.csv')

In [271]:
R_results.to_csv('R_results.csv')

## Make histograms

In [383]:
def save_dem_hist(state, folder):
    dem_wins = st_data[state]['D_actual']
    data = state_results[state]['D_wins20']
    
    fig = plt.figure(figsize=(11,11))
    
    data = np.array(data)
    d = np.diff(np.unique(data)).min()
    d_min, d_max = data.min(), data.max()
    left = d_min - float(d)/2
    right = d_max + float(d)/2
    
    plt.hist(data, bins=np.arange(left, right + d, d), color='grey',
             edgecolor='white')
    plt.axvline(x=dem_wins, color='blue')
    
    main_ticks = np.arange(d_min, d_max+1, 1)
    dem_tick = np.array([dem_wins])
    total_ticks = np.concatenate((dem_tick, main_ticks))
    
    plt.xticks(np.unique(total_ticks), fontsize=22)
    fig.supxlabel('Number of Democrat Seats Won', fontsize=24, y=0.04)
    fig.supylabel('Count', fontsize=24, x=0)
    plt.suptitle(st_data[state]['name'], size=28, y=0.94)
    plt.yticks(fontsize=22)
    
    # plt.show()
    plt.savefig(folder + '/' + state + '_Dem.png')
    plt.clf()

In [384]:
def save_rep_hist(state, folder):
    rep_wins = st_data[state]['R_actual']
    data = state_results[state]['R_wins20']
    
    fig = plt.figure(figsize=(11,11))
    
    data = np.array(data)
    d = np.diff(np.unique(data)).min()
    d_min, d_max = data.min(), data.max()
    left = d_min - float(d)/2
    right = d_max + float(d)/2
    
    plt.hist(data, bins=np.arange(left, right + d, d), color='grey',
             edgecolor='white')
    plt.axvline(x=rep_wins, color='red')
    
    main_ticks = np.arange(d_min, d_max+1, 1)
    rep_tick = np.array([rep_wins])
    total_ticks = np.concatenate((rep_tick, main_ticks))
    
    plt.xticks(np.unique(total_ticks), fontsize=22)
    fig.supxlabel('Number of Republican Seats Won', fontsize=24, y=0.04)
    fig.supylabel('Count', fontsize=24, x=0)
    plt.suptitle(st_data[state]['name'], size=28, y=0.94)
    plt.yticks(fontsize=22)
    
    # plt.show()
    plt.savefig(folder + '/' + state + '_Rep.png')
    plt.clf()

In [ ]:
for state in states:
    try:
        save_dem_hist(state, 'final_results')
        save_rep_hist(state, 'final_results')
        count += 1
        print("Saved", state)
    except Exception as e:
        print(state, '\b:', e)

## Evaluate scores

In [ ]:
for state in states:
    try:
        ex_predicted = sum(state_results[state]['D_wins20']) / len(state_results[state]['D_wins20'])
        dem = st_data[state]['D_actual']
        rep = st_data[state]['R_actual']
        total = dem + rep
        state_results[state]['score'] = ((ex_predicted - dem) / total) * 100
        state_results[state]['seat_diff'] = ex_predicted - dem
    
    except Exception as e:
        print(state, e)

In [613]:
def get_authority_avg(auth, score_type):
    index = basic_st_info.loc[basic_st_info['authority'] == auth].index
    s = 0
    c = 0
    for i in index:
        state = states[i]
        if score_type == 'eg' or score_type == 'eg_seats':
            s += abs(state_results[state][score_type])
        else:
            s += state_results[state][score_type]
        c += 1
    avg_score = s / c
    return avg_score

In [614]:
def get_authority_scores(auth, score_type):
    index = basic_st_info.loc[basic_st_info['authority'] == auth].index
    vals = []
    for i in index:
        vals.append(state_results[states[i]][score_type])
    return vals

In [619]:
def plot_scores(kind, title, color):
    split = get_authority_avg('SPLIT', kind)
    dem = get_authority_avg('DEM', kind)
    rep = get_authority_avg('REP', kind)
    irc = get_authority_avg('IRC', kind)
    
    fig = plt.figure(figsize=(12,17))
    bar = plt.bar([1, 2, 3, 4], [split, dem, rep, irc], 
                  tick_label = ['SPLIT', 'DEM', 'REP', 'IRC'], color=color)
    plt.bar_label(bar, [round(x, 2) for x in [split, dem, rep, irc]], 
                  fontsize=22, padding=10, color=color, weight="bold")
    plt.axhline(y=0, color='black')
    
    plt.xticks(fontsize=22)
    plt.yticks(fontsize=22)
    fig.supxlabel('Redistricting Authority', fontsize=25, y=0.06)
    fig.supylabel(title, fontsize=25, x=0)
    # plt.title('Average ' + title + ' of Redistricting Authorities', size=24, y=1.02)
    plt.savefig(title + "_plot.png")

In [ ]:
# example of plot_scores
plot_scores('eg', 'Efficiency Gap', 'darkgoldenrod')

In [611]:
def plot_states(kind, title):
    split = get_authority_scores('SPLIT', kind)
    dem = get_authority_scores('DEM', kind)
    rep = get_authority_scores('REP', kind)
    irc = get_authority_scores('IRC', kind)
    
    fig = plt.figure(figsize=(10,10))
    flierprops = dict(marker='o', markersize=12)
    bp = plt.boxplot([split, dem, rep, irc], flierprops=flierprops)
    
    plt.xticks(ticks=[1,2,3,4], labels=['SPLIT', 'DEM', 'REP', 'IRC'], fontsize=22)
    plt.yticks(fontsize=22)
    fig.supxlabel('Redistricting Authority', fontsize=24, y=0.02)
    fig.supylabel(title, fontsize=18, x=0)
    plt.savefig(title + "_boxplot.png")
    return bp

In [ ]:
# example of plot_states
bp = plot_states('seat_diff', 'Difference Between Expected (D) Wins and Actual Wins')